In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import PIL
import cv2
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns

# **Loading Data**

In [3]:
height = 75
width = 75

In [5]:
PATH = "/kaggle/input/eyes-rtte/"
train_dir = os.path.join(PATH)
train = keras.preprocessing.image_dataset_from_directory(train_dir,validation_split=(0.2),subset="training",seed=42,image_size=(height,width))
validation = keras.preprocessing.image_dataset_from_directory(train_dir,validation_split=(0.2),subset="validation",seed=42,image_size=(height,width))

# **Plotting Train Data**

In [16]:
plt.figure(figsize=(10,10))
for images,labels in train.take(1):
    for i in range(9):
        plt.subplot(3,3,i+1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(train.class_names[labels[i]])
        plt.axis(False)

# **Creating CNN Model**

In [17]:
scaler = keras.Sequential([keras.layers.Rescaling(1/255)])

In [18]:
augmentation = keras.Sequential([keras.layers.RandomZoom(0.2),
                                keras.layers.RandomRotation(0.3),
                                keras.layers.RandomFlip("horizontal")])

In [19]:
model = keras.Sequential([scaler,
                          keras.layers.Conv2D(16,kernel_size=3,strides=1,input_shape=(75,75,3)),
                          keras.layers.Conv2D(32,kernel_size=3,strides=1),
                          keras.layers.BatchNormalization(),
                          keras.layers.Activation("relu"),
                          keras.layers.MaxPooling2D(2),
                          keras.layers.SpatialDropout2D(0.2),
                          keras.layers.Conv2D(64,kernel_size=3,strides=1),
                          keras.layers.BatchNormalization(),
                          keras.layers.Activation("relu"),
                          keras.layers.MaxPooling2D(2),         
                          keras.layers.SpatialDropout2D(0.2),
                          keras.layers.Flatten(),
                          keras.layers.Dense(128,activation="relu"),
                          keras.layers.Dropout(0.2),
                          keras.layers.Dense(1,activation="sigmoid")

    
    
])

In [20]:
model.compile(loss="binary_crossentropy",metrics=["accuracy"],optimizer="adam")
early_stop = keras.callbacks.EarlyStopping(patience=3)

In [21]:
model.summary()

In [22]:
history = model.fit(train,validation_data=validation,epochs=10,callbacks=early_stop)

# **Evaluating Model Performance**

In [23]:
history_df = pd.DataFrame(history.history)
plt.plot(history_df.accuracy,label="Accuracy")
plt.plot(history_df.val_accuracy,label="Validation Accuracy")
plt.xlabel("Epochs"),
plt.ylabel("Accuracy")
plt.legend()
plt.grid()

In [24]:
plt.plot(history_df.loss,label="Loss")
plt.plot(history_df.val_loss,label="Validation Loss")
plt.xlabel("Epochs"),
plt.ylabel("Loss")
plt.legend()
plt.grid()

# **Making Predictions**

In [25]:
prediction = model.predict(validation)

In [26]:
prediction_labels = []
for i in prediction:
    if i >0.5:
        prediction_labels.append(1)
    else:
        prediction_labels.append(0)

In [27]:
plt.figure(figsize=(10,10))
for images,labels in validation.take(1):
    for i in range(9):
        plt.subplot(3,3,i+1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(f"Actual:{validation.class_names[labels[i]]},Predicted:{validation.class_names[prediction_labels[i]]}",fontsize=10)
        plt.axis(False)
        plt.tight_layout()


# **Second CNN Model(InceptionV3)**

In [28]:
inception = keras.applications.InceptionV3(input_shape = (75, 75, 3), include_top = False, weights = 'imagenet')

In [30]:
for layer in inception.layers:
    layer.trainable = False

In [31]:
model_inception = keras.Sequential([scaler,
                                    inception,
                                    keras.layers.Flatten(),
                                    keras.layers.Dense(30,activation="relu"),
                                    keras.layers.Dropout(0.2),
                                    keras.layers.Dense(1,activation="sigmoid")
    
])

In [ ]:
model_inception.compile(loss="binary_crossentropy",metrics=["accuracy"],optimizer="adam")
early_stop = keras.callbacks.EarlyStopping(patience=3)

In [ ]:
history_inception = model_inception.fit(train,validation_data=validation,epochs=10,callbacks=early_stop)

# **Evaluating Second Model Performance**

In [ ]:
history_inception_df = pd.DataFrame(history_inception.history)
plt.plot(history_inception_df.accuracy,label="Accuracy")
plt.plot(history_inception_df.val_accuracy,label="Validation Accuracy")
plt.xlabel("Epochs"),
plt.ylabel("Accuracy")
plt.legend()
plt.grid()

In [ ]:
plt.plot(history_inception_df.loss,label="Loss")
plt.plot(history_inception_df.val_loss,label="Validation Loss")
plt.xlabel("Epochs"),
plt.ylabel("Loss")
plt.legend()
plt.grid()

# **Third CNN Model(MobileNet)**

In [ ]:
mobile_net = keras.applications.MobileNet(input_shape=(75,75,3) , include_top=False,weights="imagenet")

In [ ]:
for layer in mobile_net.layers:
    layer.trainable = False

In [ ]:
model_mobile = keras.Sequential([scaler,
                                 mobile_net,
                                 keras.layers.Flatten(),
                                 keras.layers.Dense(30,activation="relu"),
                                 keras.layers.Dropout(0.2),
                                 keras.layers.Dense(1,activation="sigmoid")
])

In [ ]:
model_mobile.compile(loss="binary_crossentropy",metrics=["accuracy"],optimizer="adam")
early_stop = keras.callbacks.EarlyStopping(patience=3)

In [ ]:
history_mobile = model_mobile.fit(train,validation_data=validation,epochs=10,callbacks=early_stop)

In [ ]:
history_mobile_df = pd.DataFrame(history_mobile.history)
plt.plot(history_mobile_df.accuracy,label="Accuracy")
plt.plot(history_mobile_df.val_accuracy,label="Validation Accuracy")
plt.xlabel("Epochs"),
plt.ylabel("Accuracy")
plt.legend()
plt.grid()

In [ ]:
plt.plot(history_mobile_df.loss,label="Loss")
plt.plot(history_mobile_df.val_loss,label="Validation Loss")
plt.xlabel("Epochs"),
plt.ylabel("Loss")
plt.legend()
plt.grid()

# **Making Predictions With All Models**

In [ ]:
plt.figure(figsize=(15,15))
for images,labels in validation.take(1):
    for i in range(4):
        plt.subplot(2,2,i+1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.axis(False)
        
        prediction = model.predict(validation)
        prediction_2 = model_inception.predict(validation)
        prediction_3 = model_mobile.predict(validation)
        
        prediction_label = []
        prediction_2_label = []
        prediction_3_label = []
        
        for x in prediction:
            if x >0.5:
                prediction_label.append(1)
            else:
                prediction_label.append(0)
        for y in prediction_2:
            if y > 0.5:
                prediction_2_label.append(1)
            else:
                prediction_2_label.append(0)
        for z in prediction_3:
            if z > 0.5:
                prediction_3_label.append(1)
            else:
                prediction_3_label.append(0)
        
        plt.title(f"Actual {validation.class_names[labels[i]]} , CNN {validation.class_names[prediction_label[i]]}, Inception {validation.class_names[prediction_2_label[i]]}, MobileNet {validation.class_names[prediction_3_label[i]]}",fontsize=10)
    